In [ ]:
# Installing the required libraries
! pip install matplotlib

In [ ]:
import json,requests,dateutil
from datetime import datetime,timedelta
import pandas as pd
from tqdm import tqdm
from requests.auth import HTTPBasicAuth
configFile='../Data/CONFIG.json'
inputFile='../Data/input_data.csv'
fileName='../Data/hourlyData.csv'
getConfig=json.loads(open(configFile,'r').read())

In [ ]:
df= pd.read_csv(fileName)
df=df[~df['c8y_Flow_F'].isnull()]

In [ ]:
from matplotlib import pyplot
import matplotlib 
series = df['c8y_Flow_F']
series.plot()

### Training Model using Nyoka Microservice

In [ ]:
tenant_url=getConfig['c_url']
user_name, password = getConfig['c_user'],getConfig['c_pass']
data={}
stDate=df['time'][0]
START_DATE= str(datetime.strptime(stDate, "%Y-%m-%d %H:%M:%S"))
data['startDate']=START_DATE.replace(' ','T')+'+05:30'
seriesData=df['c8y_Flow_F'].values.flatten().tolist()
data['series']=seriesData
interval={}
interval['timeUnit']='HOURS'
interval['periodLength']=2
data['observationInterval']=interval
seasonality={}
seasonality['timeUnit']='HOURS'
seasonality['periodLength']=24
data['seasonality']=seasonality
time_series_data = json.dumps(data)

In [ ]:
time_series_data

In [ ]:
postURL=tenant_url+"/service/zementis/train/timeseries"
post_resp=requests.post(url=postURL,data=time_series_data,headers = {'content-type': 'application/json'},auth=(user_name, password),verify=True)

In [ ]:
post_resp.text

In [ ]:
uploaded_model=json.loads(post_resp.text)['modelName']
model_uploaded_status_url=json.loads(post_resp.text)['statusUrl']
model_uploaded_status_url

In [ ]:
tenant_url[:-1]+model_uploaded_status_url

In [ ]:
tenant_url

In [ ]:
status=True
while status:
    status_resp =requests.get(url=tenant_url+model_uploaded_status_url,auth=(user_name, password),verify=True)
    status_resp
    respStatus=json.loads(status_resp.text)['status']
    if respStatus=='SUCCESS':
        status=False
    else:
        import time
        time.sleep(10)
        print ('Training in Progress')
print ('Training Completed')

## Score

In [ ]:
url = tenant_url+'/service/zementis/apply/'+uploaded_model
prediction_response=requests.get(url = url, params = 'record={"h":12}', auth=(user_name, password), verify=True)

In [ ]:
predictedValues=pd.DataFrame([json.loads(prediction_response.text)['outputs'][0]['predicted_value_0']]).transpose()

In [ ]:
predictedValues=predictedValues.reset_index()
predictedValues.columns=['hours','c8y_Flow_F']
predictedValues['hours']=predictedValues['hours'].apply(int).apply(lambda x: x*2)
predictedValues=predictedValues.sort_values(by=['hours'],ascending=True)
predictedValues=predictedValues.reset_index()
del predictedValues['index']
predictedValues

### Predicted Hourly Consumption

In [ ]:
from matplotlib import pyplot
import matplotlib 
#%matplotlib inline 
series = predictedValues['c8y_Flow_F']
series.plot()

peak_hour_index=predictedValues['c8y_Flow_F'].to_list().index(max(predictedValues['c8y_Flow_F']))
min_hour_index=predictedValues['c8y_Flow_F'].to_list().index(min(predictedValues['c8y_Flow_F']))
print('\n The Peak Consumption observed at '+str(peak_hour_index)+' hours and its predicted value :: '+str(max(predictedValues['c8y_Flow_F'])) )
print('\n The Minimum Consumption observed at '+str(min_hour_index)+' hours and its predicted value :: '+str(min(predictedValues['c8y_Flow_F'])) )